In [1]:
import pickle, random, copy, platform, os
import numpy as np
import tensorflow as tf
import email_sending
from PIL import Image, ImageDraw

In [2]:
def img():
    with open('GAN', 'rb') as f:
        data = pickle.load(f)
        band1 = data[0]
        band2 = data[1]
        label = data[2]

    return band1, band2, label

In [3]:
#data load
band1 = img()[0]
band2 = img()[1]
y = img()[2]

In [4]:
img1 = band1.reshape(-1, 75, 75)
img2 = band2.reshape(-1, 75, 75)

In [5]:
X1 = band1.reshape(-1, 5625)
X2 = band2.reshape(-1, 5625)

In [6]:
import matplotlib.pyplot as plt

In [7]:
# expresses the label data in one-hot encoding.
n_class = 2
def onehot_encoding (Y):
    Y_onehot = np.zeros((Y.size, int(n_class)))
    for i in range(Y.size):
        Y_onehot[i][Y[i]] = 1
    return Y_onehot

In [8]:
labels = onehot_encoding(y)

In [9]:
np.amin(band1)

-45.594448

In [10]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(75, 75, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(256, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((3, 3), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((3, 3), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((3, 3), padding='same')(x)
print(encoded)
# at this point the representation is (3, 3, 64) i.e. 3*3*64-dimensional

x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((3, 3))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((3, 3))(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = UpSampling2D((3, 3))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
print(decoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Using TensorFlow backend.


Tensor("max_pooling2d_3/MaxPool:0", shape=(?, 3, 3, 64), dtype=float32)
Tensor("conv2d_7/Sigmoid:0", shape=(?, 75, 75, 1), dtype=float32)


from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [11]:
x_train, x_test = band1[:1200]/46, band1[1200:1600]/46

x_train = np.reshape(x_train, (len(x_train), 75, 75, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 75, 75, 1))  # adapt this if using `channels_first` image data format

In [12]:
from keras.callbacks import TensorBoard

autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=50,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Train on 1200 samples, validate on 400 samples
Epoch 1/10
1200/1200 [==============================] - 9s 7ms/step - loss: -5.9366 - val_loss: -7.0487
Epoch 2/10
1200/1200 [==============================] - 7s 6ms/step - loss: -7.3001 - val_loss: -7.0487
Epoch 3/10
1200/1200 [==============================] - 7s 6ms/step - loss: -7.3002 - val_loss: -7.0487
Epoch 4/10
1200/1200 [==============================] - 7s 6ms/step - loss: -7.3002 - val_loss: -7.0487
Epoch 5/10
1200/1200 [==============================] - 7s 6ms/step - loss: -7.3002 - val_loss: -7.0488
Epoch 6/10
1200/1200 [==============================] - 7s 6ms/step - loss: -7.3002 - val_loss: -7.0488
Epoch 7/10
1200/1200 [==============================] - 7s 6ms/step - loss: -7.3002 - val_loss: -7.0488
Epoch 8/10
1200/1200 [==============================] - 7s 6ms/step - loss: -7.3002 - val_loss: -7.0488
Epoch 9/10
1200/1200 [==============================] - 7s 6ms/step - loss: -7.3002 - val_loss: -7.0488
Epoch 10/10
1200/

In [13]:
decoded_imgs = autoencoder.predict(x_test)
sample_size = 10

fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

for i in range(sample_size):
    ax[0][i].set_axis_off()
    ax[1][i].set_axis_off()

    ax[0][i].imshow(x_test[i].reshape(75, 75))
    ax[1][i].imshow(decoded_imgs[i].reshape(75, 75))
    plt.gray()
plt.savefig('samples/{}.png'.format(str(1)), bbox_inches='tight')

In [14]:
plt.close(fig)